# Install Dependencies

In [ ]:
!pip install -U sagemaker transformers

# Deploy LLM Model

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'mistralai/Mistral-7B-Instruct-v0.2',
	'SM_NUM_GPUS': json.dumps(1),
	'HUGGING_FACE_HUB_TOKEN': '<REPLACE_WITH_YOUR_HF_TOKEN>'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="2.0.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.xlarge",
    endpoint_name="mistral-llm",
	container_startup_health_check_timeout=300,
  )
  

# Invoke Endpoint (Example)

In [ ]:
import boto3
import json

runtime = boto3.client("runtime.sagemaker")
LLM_ENDPOINT="mistral-llm"

response = runtime.invoke_endpoint(EndpointName=LLM_ENDPOINT, ContentType="application/json", Body=json.dumps({"inputs":f"<s>[INST]Tell me an article on Blockchain[/INST]"}).encode("utf-8"))
print(json.loads(response['Body'].read().decode('utf-8')))